# Verify training images

In [1]:
from pathlib import Path
import random

import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torchvision import transforms

from segmentation.datasets import HerbariumSheets, ImageFolder, SemiSupervisedDataLoader

/home/scman1/py_36_env/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
transform = transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomCrop((1320, 872)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor()])

target_transform = transforms.Compose([transform, transforms.Lambda(lambda x: (x * 255).long())])

batch_size = 3

# WARNING: Don't use multiple workers for loading! Doesn't work with setting random seed
train_data_labelled = HerbariumSheets(download=True, train=True, root='data', transform=transform, target_transform=target_transform)
train_loader_labelled = torch.utils.data.DataLoader(train_data_labelled, batch_size=batch_size, drop_last=True, shuffle=True)
train_data_unlabelled = ImageFolder(root='data/slides', transform=transform)
train_loader_unlabelled = torch.utils.data.DataLoader(train_data_unlabelled, batch_size=batch_size, drop_last=True, shuffle=True)
train_loader = SemiSupervisedDataLoader(train_loader_labelled, train_loader_unlabelled)

test_data_labelled = HerbariumSheets(download=True, train=False, root='data', transform=transform, target_transform=target_transform)
test_loader_labelled = torch.utils.data.DataLoader(test_data_labelled, batch_size=batch_size, drop_last=True, shuffle=True)
test_data_unlabelled = ImageFolder(root='data/slides', transform=transform)
test_loader_unlabelled = torch.utils.data.DataLoader(test_data_unlabelled, batch_size=batch_size, drop_last=True, shuffle=True)
test_loader = SemiSupervisedDataLoader(test_loader_labelled, test_loader_unlabelled)

MemoryError: 

In [ ]:
for i in range(len(train_loader.labelled.dataset)):
    if i > -1:
        training_data=train_loader.labelled.dataset[i]
        image, labels, instances = training_data
        image, labels, instances = Variable(image.unsqueeze(0)), Variable(labels.unsqueeze(0)), Variable(instances)
        # only to see images when debugging
        # see the slide
        plt.imshow(image.data[0].cpu().numpy().transpose(1,2,0))
        plt.show()
        # see the labels
        plt.imshow(labels[0,0])
        plt.show()
        # see instances
        plt.imshow(instances.cpu().numpy().squeeze())
        plt.show()
    if i == 6:
        break
        

In [ ]:
train_loader = torch.utils.data.DataLoader(test_data_labelled, batch_size=1, shuffle=True)

image, labels, instances = next(iter(train_loader))
image = Variable(image)
instances = Variable(instances + 1)
current_labels = labels[0, 0]
current_instances = instances[0]


In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 12))
for ax in axes.flatten(): ax.axis('off')

axes[0].set_title('Original image')
axes[0].imshow(image[0].data.numpy().transpose(1, 2, 0))
axes[1].set_title('Ground truth classes')
axes[1].imshow(current_labels.cpu().numpy().squeeze())
axes[2].set_title('Ground truth instances')
axes[2].imshow(current_instances.cpu().numpy().squeeze())